# Geom #3
Fully Restrained Beam with Mid-Span Point Load

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from deepxde.backend import tf
# dde.config.set_random_seed(1)

In [ ]:
soft_const = True
num_domain=100
num_boundary = 20 if soft_const == True else 0

In [ ]:
L = 1;   E = 200e9;   I = 1e-6;   P = -10000.0;   EI=E*I
sigma =0.01
geom = dde.geometry.Interval(0, L)

In [ ]:
def pde(x, y):
    d2y = dde.grad.hessian(y, x, i=0)
    d4y = dde.grad.hessian(d2y, x, i=0)
    return EI * d4y - P / (sigma * np.sqrt(2 * np.pi)) * tf.exp(-((x - L / 2) ** 2) / (2 * sigma ** 2))


def analytical_solution(x):
    y = np.zeros_like(x)
    half_L = L / 2
    mask1 = x <= half_L
    mask2 = x > half_L

    y[mask1] = (P / (48 * EI)) * (3 * L * x[mask1]**2 - 4 * x[mask1]**3)
    y[mask2] = (P / (48 * EI)) * (3 * L * (L - x[mask2])**2 - 4 * (L - x[mask2])**3)
    return y

In [ ]:
def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], L)

bcD_l = dde.icbc.DirichletBC(geom, lambda x: 0, boundary_l)
bcN_l = dde.icbc.NeumannBC(geom, lambda x: 0, boundary_l)

bcD_r = dde.icbc.DirichletBC(geom, lambda x: 0, boundary_r)
bcN_r = dde.icbc.NeumannBC(geom, lambda x: 0, boundary_r)

bcs = [bcD_l, bcD_r, bcN_l, bcN_r] if soft_const == True else []

In [ ]:
data = dde.data.PDE(geom, pde, bcs, num_domain=num_domain, num_boundary=num_boundary, solution=analytical_solution, num_test=100)

In [ ]:
net = dde.maps.FNN([1] + [50] * 3 + [1], "swish", "Glorot normal")
def output_transform(x, y):
    return x * (1 - x) * y  # Dirichlet در 0 و L enforced

if soft_const == False:
    net.apply_output_transform(output_transform)

loss_weights=[1e-14, 1, 1, 1, 1] if soft_const == True else [1e-14]

model = dde.Model(data, net)
class CustomLossWeightCallback(dde.callbacks.Callback):
    def on_epoch_end(self):
        epoch = self.model.train_state.epoch
        decay = np.exp(-0.0001 * epoch)
        self.model.loss_weights[1] = 10 * decay  # bcD_r
        self.model.loss_weights[2] = 10 * decay  # bcN_r

callback = CustomLossWeightCallback()

model.compile('adam', lr=1e-5, metrics=['l2 relative error'], loss_weights=loss_weights)
history = model.train(iterations=200000, callbacks=[callback])
model.compile('L-BFGS', loss_weights=loss_weights, metrics=['l2 relative error'])
history = model.train()

In [ ]:
X = np.linspace(0, L, 1000).reshape(-1, 1)
y_pred = model.predict(X)

y_true = analytical_solution(X.flatten())
l2_error = np.linalg.norm(y_pred.flatten() - y_true, 2) / np.linalg.norm(y_true, 2)

plt.figure(figsize=(8, 5))
plt.plot(X, y_pred, label="PINN prediction")
plt.plot(X, y_true, "--", label="Analytical solution")
plt.xlabel("x")
plt.ylabel("Deflection w(x)")
plt.title(f"Fully Restrained Beam Under Mid-Span Point Load ,L2 Relative Error: {l2_error:.2e}" )
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('mid_span_restrained_results_Ablation.png', dpi=300)
# plt.show()


In [ ]:
# Fully Restrained Beam with Mid-Span Point Load (5-run automatic)
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from deepxde.backend import tf
import random

soft_const = True
num_domain = 100
num_boundary = 20 if soft_const else 0
L = 1; E = 200e9; I = 1e-6; P = -10000.0; EI = E*I
sigma = 0.01
geom = dde.geometry.Interval(0, L)

def pde(x, y):
    d2y = dde.grad.hessian(y, x, i=0)
    d4y = dde.grad.hessian(d2y, x, i=0)
    return EI * d4y - P / (sigma * np.sqrt(2 * np.pi)) * tf.exp(-((x - L/2)**2) / (2*sigma**2))

def analytical_solution(x):
    y = np.zeros_like(x)
    half_L = L / 2
    mask1 = x <= half_L
    mask2 = x > half_L
    y[mask1] = (P / (48*EI)) * (3*L*x[mask1]**2 - 4*x[mask1]**3)
    y[mask2] = (P / (48*EI)) * (3*L*(L - x[mask2])**2 - 4*(L - x[mask2])**3)
    return y

def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], L)

# 5 seed خودکار
seeds = [42, 123, 2025, 7, 99]
l2_errors = []

for seed in seeds:
    print(f"\n=== Running with seed {seed} ===")
    np.random.seed(seed)
    dde.config.set_random_seed(seed)
    random.seed(seed)

    bcD_l = dde.icbc.DirichletBC(geom, lambda x: 0, boundary_l)
    bcN_l = dde.icbc.NeumannBC(geom, lambda x: 0, boundary_l)
    bcD_r = dde.icbc.DirichletBC(geom, lambda x: 0, boundary_r)
    bcN_r = dde.icbc.NeumannBC(geom, lambda x: 0, boundary_r)
    bcs = [bcD_l, bcD_r, bcN_l, bcN_r] if soft_const else []

    data = dde.data.PDE(geom, pde, bcs, num_domain=num_domain, num_boundary=num_boundary,
                        solution=analytical_solution, num_test=100)
    net = dde.maps.FNN([1] + [50]*3 + [1], "swish", "Glorot normal")

    def output_transform(x, y):
        return x*(1-x)*y

    if not soft_const:
        net.apply_output_transform(output_transform)

    loss_weights = [1e-14, 1, 1, 1, 1] if soft_const else [1e-14]
    model = dde.Model(data, net)

    class CustomLossWeightCallback(dde.callbacks.Callback):
        def on_epoch_end(self):
            epoch = self.model.train_state.epoch
            decay = np.exp(-0.0001*epoch)
            self.model.loss_weights[1] = 10*decay
            self.model.loss_weights[2] = 10*decay

    callback = CustomLossWeightCallback()

    model.compile('adam', lr=1e-5, metrics=['l2 relative error'], loss_weights=loss_weights)
    model.train(iterations=200000, callbacks=[callback])
    model.compile('L-BFGS', loss_weights=loss_weights, metrics=['l2 relative error'])
    model.train()

    # محاسبه L2 error
    X = np.linspace(0, L, 1000).reshape(-1, 1)
    y_pred = model.predict(X)
    y_true = analytical_solution(X.flatten())
    l2_error = np.linalg.norm(y_pred.flatten() - y_true, 2)/np.linalg.norm(y_true, 2)
    l2_errors.append(l2_error)
    print(f"L2 Relative Error for seed {seed}: {l2_error:.5e}")

# محاسبه mean ± std
mean_error = np.mean(l2_errors)
std_error = np.std(l2_errors, ddof=1)
print(f"\n=== Final L2 Relative Error (mean ± std) ===")
print(f"{mean_error:.5e} ± {std_error:.5e}")

# رسم آخرین اجرا
plt.figure(figsize=(8,5))
plt.plot(X, y_pred, label="PINN prediction")
plt.plot(X, y_true, "--", label="Analytical solution")
plt.xlabel("x")
plt.ylabel("Deflection w(x)")
plt.title(f"Fully Restrained Beam, L2 Relative Error: {mean_error:.2e} ± {std_error:.2e}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('mid_span_restrained_results_5run.png', dpi=300)
# plt.show()
